In [ ]:
%pip install pandas -q
%pip install numpy -q

In [ ]:
import pandas	as pd
import	numpy as np

In [ ]:
import pandas as pd

# 1. 최종 데이터 파일('product_data_merged.csv') 로드
try:
    product_df = pd.read_csv('product_data_merged.csv')
except FileNotFoundError:
    print("오류: 'product_data_merged.csv' 파일을 찾을 수 없습니다.")
    exit()

# 2. 피부 타입별 성분 정의
sensitive_skin_avoid = ['향료', '리모넨', '리날룰', '벤질알코올']
acne_skin_friendly = ['살리실릭애씨드', '티트리잎오일', '병풀추출물', '마데카소사이드', '나이아신아마이드']

def recommend_cosmetics(skin_type, category):
    """
    사용자의 피부 타입과 원하는 카테고리에 맞는 화장품을 추천하고 가격을 포맷팅하는 함수
    """
    
    # 카테고리로 제품 필터링
    filtered_products = product_df[product_df['카테고리'] == category].copy()
    
    recommended_list = []

    # 피부 타입에 맞는 제품 필터링
    for index, row in filtered_products.iterrows():
        if pd.isna(row['전성분']):
            continue
        ingredients = str(row['전성분']).split(';')
        
        if skin_type == 'sensitive':
            if not any(avoid in ingredients for avoid in sensitive_skin_avoid):
                recommended_list.append(row)
        
        elif skin_type == 'acne':
            if any(friendly in ingredients for friendly in acne_skin_friendly):
                recommended_list.append(row)

    if not recommended_list:
        return f"요청하신 조건(피부타입: {skin_type}, 카테고리: {category})에 맞는 제품을 찾을 수 없습니다."
        
    # --- 코드 수정 부분 ---
    # 추천 목록을 DataFrame으로 변환하고 상위 3개 선택
    result_df = pd.DataFrame(recommended_list).head(3).copy()
    
    # '가격' 컬럼이 숫자인 경우에만 포맷팅 적용
    if pd.api.types.is_numeric_dtype(result_df['가격']):
        result_df['가격'] = result_df['가격'].apply(lambda x: f'{int(x):,}원')
    
    return result_df

# --- 함수 사용 예시 ---

# 1. 민감성 피부를 위한 로션/에멀전 추천
print("--- 민감성 피부 추천 로션/에멀전 (상위 3개) ---")
sensitive_recommendations = recommend_cosmetics('sensitive', '로션/에멀전')
# '가격' 형식이 변경된 최종 결과 출력
print(sensitive_recommendations[['제품명', '브랜드명', '가격', '용량', '링크']])

print("\n" + "="*50 + "\n")

# 2. 여드름성 피부를 위한 로션/에멀전 추천
print("--- 여드름성 피부 추천 로션/에멀전 (상위 3개) ---")
acne_recommendations = recommend_cosmetics('acne', '로션/에멀전')
# '가격' 형식이 변경된 최종 결과 출력
print(acne_recommendations[['제품명', '브랜드명', '가격', '용량', '링크']])

In [ ]:
import pandas as pd

def recommend_cosmetics_by_keyword_final(user_input):
    """
    사용자 입력 키워드를 기반으로, 각 카테고리별 상위 3개 제품을 추천하는 최종 함수
    'product_data_merged.csv' 파일을 사용하며, 가격/용량/링크를 포함하고 가격 형식을 지정합니다.
    """
    try:
        # 1. 링크 정보가 포함된 최종 데이터 파일 불러오기
        product_df = pd.read_csv('product_data_merged.csv')
    except FileNotFoundError:
        return "오류: 'product_data_merged.csv' 파일을 찾을 수 없습니다."

    # 2. 긴 키워드를 먼저 인식하도록 순서 조정
    keyword_to_category = {
        '선크림': '선크림/로션', '선세럼': '선크림/로션', '선로션': '선크림/로션',
        '클렌징폼': '클렌징 폼', '클렌저': '클렌징 폼',
        '로션': '로션/에멀전', '에멀전': '로션/에멀전',
        '스킨': '스킨/토너', '토너': '스킨/토너',
        '세럼': '에센스/앰플/세럼', '에센스': '에센스/앰플/세럼', '앰플': '에센스/앰플/세럼',
        '크림': '크림',
        '마스크': '시트마스크',
        '밤': '밤/멀티밤',
    }

    recommendations = {}
    found_keywords = set()

    # 3. 사용자 입력에서 키워드를 찾아 카테고리별로 추천 목록 생성
    for keyword, category in keyword_to_category.items():
        if keyword in user_input and keyword not in found_keywords:
            # 해당 카테고리의 제품들을 찾아 상위 3개 선택
            matched_products = product_df[product_df['카테고리'] == category].head(3).copy()
            
            # --- 가격 포맷팅 및 정보 추가 ---
            if not matched_products.empty:
                # '가격' 컬럼이 숫자인 경우에만 포맷팅 적용
                if pd.api.types.is_numeric_dtype(matched_products['가격']):
                     matched_products['가격'] = matched_products['가격'].apply(lambda x: f'{int(x):,}원')
                
                # 필요한 컬럼만 선택하여 저장
                recommendations[category] = matched_products[['제품명', '브랜드명', '가격', '용량', '링크']]

                # 하위 키워드 중복 추천 방지
                for k in keyword_to_category:
                    if k in keyword:
                        found_keywords.add(k)

    # 4. 최종 결과 출력
    if not recommendations:
        return "추천해드릴 제품을 찾지 못했어요. 다른 제품 종류를 말씀해주시겠어요?"

    response_text = "요청하신 제품들을 찾아봤어요! 😊\n"
    for category, products_df in recommendations.items():
        response_text += f"\n--- {category} 추천 (상위 3개) ---\n"
        # to_string()을 사용하여 데이터프레임을 예쁜 문자열로 변환
        response_text += products_df.to_string(index=False)
        response_text += "\n"
        
    return response_text

# --- 챗봇 시뮬레이션 예시 ---

# 예시 1: "로션" 키워드가 포함된 경우
user_message_1 = "요즘 피부가 건조해서 그런데, 순한 로션 좀 추천해줄래?"
recommendations_1 = recommend_cosmetics_by_keyword_final(user_message_1)
print(f"사용자 입력: \"{user_message_1}\"\n")
print(recommendations_1)
print("\n" + "="*80 + "\n")

# 예시 2: "선크림", "클렌저" 키워드가 모두 포함된 경우
user_message_2 = "여름에 쓸 선크림이랑 클렌저 같이 보여줘"
recommendations_2 = recommend_cosmetics_by_keyword_final(user_message_2)
print(f"사용자 입력: \"{user_message_2}\"\n")
print(recommendations_2)

# 예시 3: 추천 제품이 없는 경우
user_message_3 = "아이라이너는 뭐가 좋아?"
recommendations_3 = recommend_cosmetics_by_keyword(user_message_3)
print(f"사용자 입력: \"{user_message_3}\"")
print("--- 추천 결과 ---")
print(recommendations_3)

In [ ]:
import pandas as pd
import numpy as np

# --- 데이터 파일 로드 ---
try:
    product_df = pd.read_csv('product_data_merged.csv')
    ingredient_df = pd.read_csv('ingredient_data.csv')
except FileNotFoundError:
    print("오류: 'product_data_merged.csv' 또는 'ingredient_data.csv' 파일을 찾을 수 없습니다.")
    exit()

# --- EWG 등급 데이터를 빠르게 찾기 위한 딕셔너리 생성 ---
# EWG등급이 없는 경우(NaN)를 제외하고, 등급이 없는 성분은 중립적인 3등급으로 처리
ingredient_df.dropna(subset=['EWG등급'], inplace=True)
ewg_dict = pd.Series(ingredient_df['EWG등급'].values, index=ingredient_df['한국어성분명']).to_dict()

# --- 1. 피부 고민 및 타입별 성분 리스트 정의 ---

# 효능 성분 (피부 고민별 가산점)
efficacy_ingredients = {
    '진정': ['병풀추출물', '마데카소사이드', '판테놀', '알란토인', '약모밀추출물'],
    '미백': ['나이아신아마이드', '알부틴', '비타민C', '글루타티온'],
    '보습': ['히알루론산', '글리세린', '세라마이드엔피', '스쿠알란', '베타-글루칸']
}

# 주의 성분 (피부 타입별 감점)
caution_ingredients = {
    '민감성': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올'],
    '지성': ['미네랄오일', '시어버터', '코코넛야자오일', '다이메티콘'],
    '건성': ['에탄올', '변성알코올', '살리실릭애씨드']
}


def calculate_product_score(ingredients_str, skin_concern, skin_type):
    """
    하나의 제품에 대한 최종 점수를 계산하는 함수
    """
    if pd.isna(ingredients_str):
        return 0

    ingredients = ingredients_str.split(';')
    
    # --- 2. EWG 안전성 점수 계산 ---
    ewg_scores = []
    for ing in ingredients:
        # ewg_dict에 성분이 없으면 3등급으로 간주, 있으면 해당 등급 사용
        grade = ewg_dict.get(ing, 3.0) 
        # 역방향 점수 계산: 11 - 등급
        reversed_score = 11 - grade
        ewg_scores.append(reversed_score)
    
    # EWG 점수 평균 계산 (성분이 하나도 없는 경우 0점 처리)
    avg_ewg_score = np.mean(ewg_scores) if ewg_scores else 0

    # --- 3. 효능 점수 (가산점) 계산 ---
    efficacy_score = 0
    if skin_concern in efficacy_ingredients:
        for ing in ingredients:
            if ing in efficacy_ingredients[skin_concern]:
                efficacy_score += 5  # 해당 성분 발견 시 +5점

    # --- 4. 주의 성분 (감점) 계산 ---
    caution_score = 0
    if skin_type in caution_ingredients:
        for ing in ingredients:
            if ing in caution_ingredients[skin_type]:
                caution_score -= 10 # 해당 성분 발견 시 -10점

    # --- 최종 점수 합산 ---
    final_score = avg_ewg_score + efficacy_score + caution_score
    return final_score


def recommend_top_products(skin_concern, skin_type):
    """
    사용자의 피부 고민과 타입에 맞춰 제품을 추천하는 메인 함수
    """
    
    # 모든 제품에 대해 최종 점수 계산하여 새로운 컬럼에 추가
    product_df['최종_점수'] = product_df['전성분'].apply(
        lambda x: calculate_product_score(x, skin_concern, skin_type)
    )
    
    # 최종 점수가 높은 순으로 정렬하여 상위 3개 제품 선택
    top_3_products = product_df.sort_values(by='최종_점수', ascending=False).head(3).copy()
    
    # 가독성을 위해 점수는 소수점 둘째 자리까지만 표시
    top_3_products['최종_점수'] = top_3_products['최종_점수'].round(2)
    
    # 가격 포맷팅
    if pd.api.types.is_numeric_dtype(top_3_products['가격']):
        top_3_products['가격'] = top_3_products['가격'].apply(lambda x: f'{int(x):,}원')

    return top_3_products[['제품명', '브랜드명', '최종_점수', '가격', '링크']]

# --- 함수 사용 예시 ---

# 예시: 피부 고민이 '진정'이고, 피부 타입이 '민감성'인 사용자를 위한 추천
skin_concern_input = '진정'
skin_type_input = '민감성'

print(f"✅ 피부 고민: '{skin_concern_input}', 피부 타입: '{skin_type_input}' 맞춤 추천 Top 3\n")
recommendations = recommend_top_products(skin_concern_input, skin_type_input)

# to_string()을 사용하여 DataFrame을 깔끔한 텍스트로 출력
print(recommendations.to_string(index=False))

In [ ]:
import pandas as pd
import numpy as np

def calculate_final_scores(user_skin_concern, user_skin_type):
    """
    사용자의 피부 정보(고민, 타입)를 바탕으로 모든 제품의 최종 점수를 계산하는 함수.

    Args:
        user_skin_concern (str): 사용자가 선택한 피부 고민 (예: '진정').
        user_skin_type (str): 사용자가 선택한 피부 타입 (예: '민감성').

    Returns:
        pandas.DataFrame: 각 제품에 '최종_점수'가 추가된 데이터프레임.
                          오류 발생 시 None을 반환.
    """
    try:
        # 1. 데이터 파일 로드
        product_df = pd.read_csv('product_with_score.csv')
        ingredient_df = pd.read_csv('ingredient_data.csv')
    except FileNotFoundError:
        print("오류: 'product_with_score.csv' 또는 'ingredient_data.csv' 파일을 찾을 수 없습니다.")
        return None

    # 2. 데이터 전처리 및 준비
    ingredient_df.dropna(subset=['EWG등급'], inplace=True)
    ewg_dict = pd.Series(ingredient_df['EWG등급'].values, index=ingredient_df['한국어성분명']).to_dict()

    # 효능 성분 리스트
    efficacy_ingredients = {
        '진정': ['병풀추출물', '마데카소사이드', '판테놀', '알란토인', '약모밀추출물'],
        '미백': ['나이아신아마이드', '알부틴', '비타민C', '글루타티온'],
        '보습': ['소듐하이알루로네이트', '글리세린', '세라마이드엔피', '스쿠알란', '베타-글루칸']
    }
    
    # 피부 타입별 주의 성분 리스트
    caution_ingredients = {
        '민감성': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올'],
        '지성': ['미네랄오일', '시어버터', '코코넛야자일', '다이메티콘'],
        '건성': ['에탄올', '변성알코올', '살리실릭애씨드']
    }

    # 3. 각 제품의 최종 점수를 계산하는 내부 함수
    def get_score_for_product(row):
        ingredients_str = row['전성분']
        harmfulness_score = row['유해성_점수']

        if pd.isna(ingredients_str):
            return -np.inf # 점수를 최하점으로 만들어 추천되지 않도록 함

        ingredients = ingredients_str.split(';')

        # --- EWG 안전성 점수 계산 ---
        ewg_scores = [11 - ewg_dict.get(ing, 3.0) for ing in ingredients]
        avg_ewg_score = np.mean(ewg_scores) if ewg_scores else 0

        # --- 효능 점수 계산 ---
        efficacy_score = 0
        if user_skin_concern in efficacy_ingredients:
            efficacy_score = sum(1 for ing in ingredients if ing in efficacy_ingredients[user_skin_concern])

        # --- 피부 타입별 주의 성분 감점 계산 ---
        caution_penalty = 0
        if user_skin_type in caution_ingredients:
            caution_penalty = sum(2 for ing in ingredients if ing in caution_ingredients[user_skin_type])
        
        # --- 최종 점수 합산 ---
        final_score = avg_ewg_score + efficacy_score - harmfulness_score - caution_penalty
        return final_score

    # 4. 모든 제품에 점수 계산 함수를 적용
    product_df['최종_점수'] = product_df.apply(get_score_for_product, axis=1)

    print("\n✅ 사용자 맞춤 점수 계산이 완료되었습니다.")
    return product_df

def main():
    """
    사용자로부터 정보를 입력받아 맞춤 화장품을 추천하는 메인 프로그램
    """
    # --- 사용자 정보 입력 ---
    print("--- 당신에게 꼭 맞는 화장품을 찾아드립니다! ---")

    # 1. 피부 고민 입력
    concerns = ['진정', '미백', '보습']
    print("\n1. 어떤 피부 고민을 가지고 계신가요?:")
    for i, concern in enumerate(concerns):
        print(f"  {i+1}. {concern}")
    
    while True:
        try:
            choice = int(input("번호를 선택해주세요: "))
            if 1 <= choice <= len(concerns):
                user_concern = concerns[choice-1]
                break
            else:
                print("1~3 사이의 번호를 입력해주세요.")
        except ValueError:
            print("유효한 번호를 입력해주세요.")

    # 2. 피부 타입 입력
    types = ['민감성', '지성', '건성']
    print("\n2. 당신의 피부 타입은 무엇인가요?:")
    for i, t in enumerate(types):
        print(f"  {i+1}. {t}")

    while True:
        try:
            choice = int(input("번호를 선택해주세요: "))
            if 1 <= choice <= len(types):
                user_type = types[choice-1]
                break
            else:
                print("1~3 사이의 번호를 입력해주세요.")
        except ValueError:
            print("유효한 번호를 입력해주세요.")

    # --- 점수 계산 및 추천 실행 ---
    scored_df = calculate_final_scores(user_concern, user_type)

    if scored_df is not None:
        # 점수가 높은 순으로 정렬 후 상위 5개 제품 추천
        top_5_products = scored_df.sort_values(by='최종_점수', ascending=False).head(5)
        
        print(f"\n--- '{user_concern}' 및 '{user_type}' 피부를 위한 화장품 추천 Top 5 ---")
        print(top_5_products[['제품명', '브랜드명', '최종_점수']].to_string(index=False))


if __name__ == '__main__':
    main()



In [ ]:
import pandas as pd
import numpy as np

def calculate_final_scores(user_skin_concerns, user_skin_type):
    """
    사용자의 피부 정보(고민-리스트, 타입)를 바탕으로 모든 제품의 최종 점수를 계산하는 함수.

    Args:
        user_skin_concerns (list): 사용자가 선택한 피부 고민 목록 (예: ['진정', '보습']).
        user_skin_type (str): 사용자가 선택한 피부 타입 (예: '민감성').

    Returns:
        pandas.DataFrame: 각 제품에 '최종_점수'가 추가된 데이터프레임.
    """
    try:
        product_df = pd.read_csv('product_with_score.csv')
        ingredient_df = pd.read_csv('ingredient_data.csv')
    except FileNotFoundError:
        print("오류: 'product_with_score.csv' 또는 'ingredient_data.csv' 파일을 찾을 수 없습니다.")
        return None

    ingredient_df.dropna(subset=['EWG등급'], inplace=True)
    ewg_dict = pd.Series(ingredient_df['EWG등급'].values, index=ingredient_df['한국어성분명']).to_dict()

    efficacy_ingredients = {
        '진정': ['병풀추출물', '마데카소사이드', '판테놀', '알란토인', '약모밀추출물'],
        '미백': ['나이아신아마이드', '알부틴', '비타민C', '글루타티온'],
        '보습': ['소듐하이알루로네이트', '글리세린', '세라마이드엔피', '스쿠알란', '베타-글루칸'],
        '주름/탄력': ['아데노신', '아세틸헥사펩타이드-8', '카퍼트라이펩타이드-1', '팔미토일펜타펩타이드-4', '하이드롤라이즈드콜라겐'],
        '모공/피지': ['살리실릭애씨드', '티트리잎오일', '버지니아풍년화추출물', '나이아신아마이드', '카프릴로일살리실릭애씨드']
    }
    
    caution_ingredients = {
        '민감성': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올'],
        '지성': ['미네랄오일', '시어버터', '코코넛야자일', '다이메티콘'],
        '건성': ['에탄올', '변성알코올', '살리실릭애씨드']
    }

    def get_score_for_product(row):
        ingredients_str = row['전성분']
        harmfulness_score = row['유해성_점수']

        if pd.isna(ingredients_str):
            return -np.inf

        ingredients = ingredients_str.split(';')
        
        ewg_scores = [11 - ewg_dict.get(ing, 3.0) for ing in ingredients]
        avg_ewg_score = np.mean(ewg_scores) if ewg_scores else 0

        efficacy_score = 0
        for concern in user_skin_concerns:
            if concern in efficacy_ingredients:
                efficacy_score += sum(1 for ing in ingredients if ing in efficacy_ingredients[concern])

        caution_penalty = 0
        if user_skin_type in caution_ingredients:
            caution_penalty = sum(2 for ing in ingredients if ing in caution_ingredients[user_skin_type])
        
        final_score = avg_ewg_score + efficacy_score - harmfulness_score - caution_penalty
        return final_score

    product_df['최종_점수'] = product_df.apply(get_score_for_product, axis=1)
    return product_df

def recommend_by_selection(scored_df):
    """선택지 기반 추천 결과를 출력하는 함수"""
    top_5_products = scored_df.sort_values(by='최종_점수', ascending=False).head(5)
    print("\n--- 🕵️‍♀️ 당신을 위한 맞춤 화장품 추천 Top 5 ---")
    print(top_5_products[['제품명', '브랜드명', '카테고리', '최종_점수']].to_string(index=False))

def recommend_by_chatbot(scored_df, user_input):
    """챗봇 입력 기반 추천 결과를 출력하는 함수"""
    keyword_to_category = {
        '선크림': '선크림/로션', '선세럼': '선크림/로션', '선로션': '선크림/로션',
        '클렌징폼': '클렌징 폼', '클렌저': '클렌징 폼',
        '로션': '로션/에멀전', '에멀전': '로션/에멀전',
        '스킨': '스킨/토너', '토너': '스킨/토너',
        '세럼': '에센스/앰플/세럼', '에센스': '에센스/앰플/세럼', '앰플': '에센스/앰플/세럼',
        '크림': '크림', '마스크': '시트마스크', '밤': '밤/멀티밤'
    }

    found_category = None
    for keyword, category in keyword_to_category.items():
        if keyword in user_input:
            found_category = category
            break
    
    if not found_category:
        print("\n죄송합니다. 어떤 종류의 제품을 찾으시는지 파악하기 어려워요.\n('크림', '로션', '선크림' 등) 제품 종류를 명확히 말씀해주시겠어요?")
        return

    category_products = scored_df[scored_df['카테고리'] == found_category]
    top_3_products = category_products.sort_values(by='최종_점수', ascending=False).head(3)

    print(f"\n--- 🤖 챗봇이 추천하는 '{found_category}' Top 3 ---")
    print(top_3_products[['제품명', '브랜드명', '최종_점수']].to_string(index=False))

def main():
    """메인 프로그램 실행 함수"""
    print("--- 당신에게 꼭 맞는 화장품을 찾아드립니다! ---")

    types = ['민감성', '지성', '건성']
    print("\n먼저, 당신의 피부 타입을 알려주세요:")
    for i, t in enumerate(types):
        print(f"  {i+1}. {t}")
    while True:
        try:
            choice = int(input("번호를 선택해주세요: "))
            if 1 <= choice <= len(types):
                user_type = types[choice-1]
                break
            else:
                print(f"1~{len(types)} 사이의 번호를 입력해주세요.")
        except ValueError:
            print("유효한 번호를 입력해주세요.")
    
    print(f"\n✅ 당신의 피부 타입은 '{user_type}'이군요!")
    print("-" * 40)
    
    while True:
        print("\n어떤 방식으로 추천을 도와드릴까요?")
        print("  1. 선택지로 추천받기 (피부 고민 중복 선택 가능)")
        print("  2. 챗봇에게 자유롭게 물어보기")
        print("  3. 종료하기")
        
        try:
            main_choice = int(input("번호를 선택해주세요: "))
            
            if main_choice == 1:
                concerns = ['진정', '미백', '보습', '주름/탄력', '모공/피지']
                print("\n어떤 피부 고민을 해결하고 싶으신가요? (여러 개 선택 가능)")
                for i, concern in enumerate(concerns):
                    print(f"  {i+1}. {concern}")
                
                user_concerns = []
                while True:
                    try:
                        choices_str = input("원하는 고민의 번호를 모두 골라주세요 (예: 1 3): ")
                        choices = [int(c) for c in choices_str.split()]
                        
                        valid_choices = True
                        temp_concerns = []
                        for c in choices:
                            if 1 <= c <= len(concerns):
                                temp_concerns.append(concerns[c-1])
                            else:
                                print(f"'{c}'는 잘못된 번호입니다. 1~{len(concerns)} 사이의 번호만 입력해주세요.")
                                valid_choices = False
                                break
                        
                        if valid_choices:
                            user_concerns = list(set(temp_concerns)) # 중복 제거
                            break
                    except ValueError:
                        print("숫자와 공백만 사용하여 번호를 입력해주세요. (예: 1 3)")
                
                print("\n점수를 계산 중입니다. 잠시만 기다려주세요...")
                scored_df = calculate_final_scores(user_concerns, user_type)
                if scored_df is not None:
                    recommend_by_selection(scored_df)

            elif main_choice == 2:
                user_input = input("\n찾고 있는 화장품에 대해 자유롭게 말씀해주세요: ")
                # 챗봇 모드에서는 모든 고민을 종합적으로 고려
                all_concerns = ['진정', '미백', '보습', '주름/탄력', '모공/피지']
                print("\n점수를 계산 중입니다. 잠시만 기다려주세요...")
                scored_df = calculate_final_scores(all_concerns, user_type)
                if scored_df is not None:
                    recommend_by_chatbot(scored_df, user_input)

            elif main_choice == 3:
                print("\n프로그램을 종료합니다. 이용해주셔서 감사합니다!")
                break
            else:
                print("1, 2, 3 중 하나를 선택해주세요.")

        except ValueError:
            print("유효한 번호를 입력해주세요.")

if __name__ == '__main__':
    main()



In [ ]:
import pandas as pd
import numpy as np

def calculate_final_scores(user_skin_concerns, user_skin_type):
    """
    사용자의 피부 정보(고민-리스트, 타입)를 바탕으로 모든 제품의 최종 점수를 계산하는 함수.
    이 함수는 추천 로직의 핵심 알고리즘 역할을 합니다.

    Args:
        user_skin_concerns (list): 사용자가 선택한 피부 고민 목록 (예: ['진정', '보습']).
        user_skin_type (str): 사용자가 선택한 피부 타입 (예: '민감성').

    Returns:
        pandas.DataFrame: 각 제품에 '최종_점수'가 추가된 데이터프레임.
                          오류 발생 시 None을 반환.
    """
    try:
        # 1. 데이터 파일 로드
        product_df = pd.read_csv('product_with_score.csv')
        ingredient_df = pd.read_csv('ingredient_data.csv')
    except FileNotFoundError:
        print("오류: 'product_with_score.csv' 또는 'ingredient_data.csv' 파일을 찾을 수 없습니다.")
        return None

    # 2. 데이터 전처리 및 준비
    # EWG 등급이 없는 성분은 제외하고, 빠르게 찾을 수 있도록 딕셔너리로 변환
    ingredient_df.dropna(subset=['EWG등급'], inplace=True)
    ewg_dict = pd.Series(ingredient_df['EWG등급'].values, index=ingredient_df['한국어성분명']).to_dict()

    # 효능 성분 리스트 (피부 고민별 가산점)
    efficacy_ingredients = {
        '진정': ['병풀추출물', '마데카소사이드', '판테놀', '알란토인', '약모밀추출물'],
        '미백': ['나이아신아마이드', '알부틴', '비타민C', '글루타티온'],
        '보습': ['소듐하이알루로네이트', '글리세린', '세라마이드엔피', '스쿠알란', '베타-글루칸'],
        '주름/탄력': ['아데노신', '아세틸헥사펩타이드-8', '카퍼트라이펩타이드-1', '팔미토일펜타펩타이드-4', '하이드롤라이즈드콜라겐'],
        '모공/피지': ['살리실릭애씨드', '티트리잎오일', '버지니아풍년화추출물', '나이아신아마이드', '카프릴로일살리실릭애씨드']
    }
    
    # 피부 타입별 주의 성분 리스트 (감점)
    caution_ingredients = {
        '민감성': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올'],
        '지성': ['미네랄오일', '시어버터', '코코넛야자일', '다이메티콘'],
        '건성': ['에탄올', '변성알코올', '살리실릭애씨드']
    }

    # 3. 각 제품의 최종 점수를 계산하는 내부 함수
    def get_score_for_product(row):
        ingredients_str = row['전성분']
        harmfulness_score = row['유해성_점수']

        # 전성분 정보가 없으면 추천되지 않도록 최하점 부여
        if pd.isna(ingredients_str):
            return -np.inf

        ingredients = ingredients_str.split(';')
        
        # --- EWG 안전성 점수 계산 ---
        # 11에서 등급을 빼서 역방향 점수로 변환 후, 성분들의 평균 점수를 계산
        ewg_scores = [11 - ewg_dict.get(ing, 3.0) for ing in ingredients]
        avg_ewg_score = np.mean(ewg_scores) if ewg_scores else 0

        # --- 효능 점수 계산 ---
        # 선택된 모든 피부 고민에 대해 효능 성분이 있으면 개수마다 +1점
        efficacy_score = 0
        for concern in user_skin_concerns:
            if concern in efficacy_ingredients:
                efficacy_score += sum(1 for ing in ingredients if ing in efficacy_ingredients[concern])

        # --- 피부 타입별 주의 성분 감점 계산 ---
        # 주의 성분이 있으면 개수마다 -2점
        caution_penalty = 0
        if user_skin_type in caution_ingredients:
            caution_penalty = sum(2 for ing in ingredients if ing in caution_ingredients[user_skin_type])
        
        # --- 최종 점수 합산 ---
        final_score = avg_ewg_score + efficacy_score - harmfulness_score - caution_penalty
        return final_score

    # 4. 모든 제품에 점수 계산 함수를 적용하여 '최종_점수' 컬럼 추가
    product_df['최종_점수'] = product_df.apply(get_score_for_product, axis=1)
    
    print("\n✅ 사용자 맞춤 점수 계산이 완료되었습니다.")
    return product_df

# # --- 이 파일을 직접 실행했을 때 테스트를 위한 예시 코드 ---
# # 다른 파일에서 이 함수를 import해서 사용할 때는 아래 코드가 실행되지 않습니다.
# if __name__ == '__main__':
#     # 테스트 케이스: '진정'과 '보습' 고민이 있는 '민감성' 피부
#     test_concerns = ['진정', '보습']
#     test_type = '민감성'
    
#     print(f"테스트 케이스: 피부고민={test_concerns}, 피부타입='{test_type}'")
    
#     # 점수 계산 함수 실행
#     scored_dataframe = calculate_final_scores(test_concerns, test_type)
    
#     # 결과 확인
#     if scored_dataframe is not None:
#         # 점수가 높은 순으로 정렬하여 상위 5개 제품 정보 출력
#         top_5 = scored_dataframe.sort_values(by='최종_점수', ascending=False).head(5)
#         print("\n--- 점수 계산 결과 (상위 5개) ---")
#         print(top_5[['제품명', '브랜드명', '최종_점수']])


In [ ]:
import pandas as pd
import numpy as np

# --- 데이터 전역 로드 (효율성 개선) ---
# 프로그램 시작 시 한 번만 파일을 불러옵니다.
try:
    PRODUCT_DF = pd.read_csv('product_with_score.csv')
    INGREDIENT_DF = pd.read_csv('ingredient_data.csv')
    
    # EWG 등급 딕셔너리 미리 생성
    INGREDIENT_DF.dropna(subset=['EWG등급'], inplace=True)
    EWG_DICT = pd.Series(INGREDIENT_DF['EWG등급'].values, index=INGREDIENT_DF['한국어성분명']).to_dict()
    
except FileNotFoundError:
    print("오류: 'product_with_score.csv' 또는 'ingredient_data.csv' 파일을 찾을 수 없습니다.")
    print("프로그램을 종료합니다.")
    PRODUCT_DF = None # 오류 발생 시 DataFrame을 None으로 설정

# --- 성분 리스트 정의 ---
EFFICACY_INGREDIENTS = {
    '진정': ['병풀추출물', '마데카소사이드', '판테놀', '알란토인', '약모밀추출물'],
    '미백': ['나이아신아마이드', '알부틴', '비타민C', '글루타티온'],
    '보습': ['소듐하이알루로네이트', '글리세린', '세라마이드엔피', '스쿠알란', '베타-글루칸'],
    '주름/탄력': ['아데노신', '아세틸헥사펩타이드-8', '카퍼트라이펩타이드-1', '팔미토일펜타펩타이드-4', '하이드롤라이즈드콜라겐'],
    '모공/피지': ['살리실릭애씨드', '티트리잎오일', '버지니아풍년화추출물', '나이아신아마이드', '카프릴로일살리실릭애씨드']
}

CAUTION_INGREDIENTS = {
    '민감성': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올'],
    '지성': ['미네랄오일', '시어버터', '코코넛야자일', '다이메티콘'],
    '건성': ['에탄올', '변성알코올', '살리실릭애씨드']
}


def calculate_final_scores(product_df, user_skin_concerns, user_skin_type):
    """
    사용자의 피부 정보(고민-리스트, 타입)를 바탕으로 모든 제품의 최종 점수를 계산하는 함수.
    """
    
    def get_score_for_product(row):
        ingredients_str = row['전성분']
        harmfulness_score = row['유해성_점수']

        if pd.isna(ingredients_str):
            return -np.inf

        ingredients = ingredients_str.split(';')
        
        # EWG 안전성 점수
        ewg_scores = [11 - EWG_DICT.get(ing, 3.0) for ing in ingredients]
        avg_ewg_score = np.mean(ewg_scores) if ewg_scores else 0

        # 효능 점수
        efficacy_score = 0
        for concern in user_skin_concerns:
            if concern in EFFICACY_INGREDIENTS:
                efficacy_score += sum(1 for ing in ingredients if ing in EFFICACY_INGREDIENTS[concern])

        # 주의 성분 감점
        caution_penalty = 0
        if user_skin_type in CAUTION_INGREDIENTS:
            caution_penalty = sum(2 for ing in ingredients if ing in CAUTION_INGREDIENTS[user_skin_type])
        
        # 최종 점수 합산
        final_score = avg_ewg_score + efficacy_score - harmfulness_score - caution_penalty
        return final_score

    # 원본 DataFrame을 복사하여 점수 계산 진행
    df_copy = product_df.copy()
    df_copy['최종_점수'] = df_copy.apply(get_score_for_product, axis=1)
    return df_copy

def recommend_by_selection(scored_df, selected_category):
    """선택지 기반 추천 결과를 출력하는 함수"""
    category_products = scored_df[scored_df['카테고리'] == selected_category]
    top_5_products = category_products.sort_values(by='최종_점수', ascending=False).head(5)
    
    if top_5_products.empty:
        print(f"\n죄송합니다. '{selected_category}' 카테고리에서 적합한 추천 제품을 찾지 못했습니다.")
        return
        
    print(f"\n--- 🕵️‍♀️ 당신을 위한 맞춤 '{selected_category}' 추천 Top 5 ---")
    print(top_5_products[['제품명', '브랜드명', '최종_점수']].to_string(index=False))

def recommend_by_chatbot(scored_df, user_input):
    """챗봇 입력 기반 추천 결과를 출력하는 함수"""
    keyword_to_category = {
        '선크림': '선크림/로션', '선세럼': '선크림/로션', '선로션': '선크림/로션',
        '클렌징폼': '클렌징 폼', '클렌저': '클렌징 폼',
        '로션': '로션/에멀전', '에멀전': '로션/에멀전',
        '스킨': '스킨/토너', '토너': '스킨/토너',
        '세럼': '에센스/앰플/세럼', '에센스': '에센스/앰플/세럼', '앰플': '에센스/앰플/세럼',
        '크림': '크림', '마스크': '시트마스크', '밤': '밤/멀티밤'
    }

    found_category = None
    for keyword, category in keyword_to_category.items():
        if keyword in user_input:
            found_category = category
            break
    
    if not found_category:
        print("\n죄송합니다. 어떤 종류의 제품을 찾으시는지 파악하기 어려워요.\n('크림', '로션', '선크림' 등) 제품 종류를 명확히 말씀해주시겠어요?")
        return

    category_products = scored_df[scored_df['카테고리'] == found_category]
    top_3_products = category_products.sort_values(by='최종_점수', ascending=False).head(3)

    if top_3_products.empty:
        print(f"\n죄송합니다. '{found_category}' 카테고리에서 적합한 추천 제품을 찾지 못했습니다.")
        return

    print(f"\n--- 🤖 챗봇이 추천하는 '{found_category}' Top 3 ---")
    print(top_3_products[['제품명', '브랜드명', '최종_점수']].to_string(index=False))

def main():
    """메인 프로그램 실행 함수"""
    # 프로그램 시작 시 데이터 로드에 실패했으면 실행하지 않음
    if PRODUCT_DF is None:
        return
        
    print("--- 당신에게 꼭 맞는 화장품을 찾아드립니다! ---")

    types = ['민감성', '지성', '건성']
    print("\n먼저, 당신의 피부 타입을 알려주세요:")
    for i, t in enumerate(types):
        print(f"  {i+1}. {t}")
    while True:
        try:
            choice = int(input("번호를 선택해주세요: "))
            if 1 <= choice <= len(types):
                user_type = types[choice-1]
                break
            else:
                print(f"1~{len(types)} 사이의 번호를 입력해주세요.")
        except ValueError:
            print("유효한 번호를 입력해주세요.")
    
    print(f"\n✅ 당신의 피부 타입은 '{user_type}'이군요!")
    print("-" * 40)
    
    while True:
        print("\n어떤 방식으로 추천을 도와드릴까요?")
        print("  1. 선택지로 추천받기 (피부 고민, 카테고리 선택)")
        print("  2. 챗봇에게 자유롭게 물어보기")
        print("  3. 종료하기")
        
        try:
            main_choice = int(input("번호를 선택해주세요: "))
            
            if main_choice == 1:
                concerns = list(EFFICACY_INGREDIENTS.keys())
                print("\n어떤 피부 고민을 해결하고 싶으신가요? (여러 개 선택 가능)")
                for i, concern in enumerate(concerns):
                    print(f"  {i+1}. {concern}")
                
                user_concerns = []
                while True:
                    try:
                        choices_str = input("원하는 고민의 번호를 모두 골라주세요 (예: 1 3): ")
                        choices = [int(c) for c in choices_str.split()]
                        
                        valid_choices = True
                        temp_concerns = []
                        for c in choices:
                            if 1 <= c <= len(concerns):
                                temp_concerns.append(concerns[c-1])
                            else:
                                print(f"'{c}'는 잘못된 번호입니다. 1~{len(concerns)} 사이의 번호만 입력해주세요.")
                                valid_choices = False
                                break
                        
                        if valid_choices and temp_concerns:
                            user_concerns = list(set(temp_concerns))
                            break
                        elif not temp_concerns:
                            print("하나 이상의 고민을 선택해주세요.")
                    except ValueError:
                        print("숫자와 공백만 사용하여 번호를 입력해주세요. (예: 1 3)")
                
                print(f"\n✅ 선택된 고민: {', '.join(user_concerns)}")
                print("\n점수를 계산 중입니다. 잠시만 기다려주세요...")
                scored_df = calculate_final_scores(PRODUCT_DF, user_concerns, user_type)
                
                all_categories = sorted(PRODUCT_DF['카테고리'].unique().tolist())
                print("\n어떤 종류의 제품을 찾으시나요?")
                for i, category in enumerate(all_categories):
                    print(f"  {i+1}. {category}")
                
                while True:
                    try:
                        category_choice = int(input("번호를 선택해주세요: "))
                        if 1 <= category_choice <= len(all_categories):
                            selected_category = all_categories[category_choice - 1]
                            break
                        else:
                            print(f"1~{len(all_categories)} 사이의 번호를 입력해주세요.")
                    except ValueError:
                        print("유효한 번호를 입력해주세요.")
                
                recommend_by_selection(scored_df, selected_category)

            elif main_choice == 2:
                user_input = input("\n찾고 있는 화장품에 대해 자유롭게 말씀해주세요: ")
                all_concerns = list(EFFICACY_INGREDIENTS.keys())
                print("\n점수를 계산 중입니다. 잠시만 기다려주세요...")
                scored_df = calculate_final_scores(PRODUCT_DF, all_concerns, user_type)
                recommend_by_chatbot(scored_df, user_input)

            elif main_choice == 3:
                print("\n프로그램을 종료합니다. 이용해주셔서 감사합니다!")
                break
            else:
                print("1, 2, 3 중 하나를 선택해주세요.")

        except ValueError:
            print("유효한 번호를 입력해주세요.")

if __name__ == '__main__':
    main()



In [ ]:
import pandas as pd
import numpy as np

# --- 데이터 전역 로드 (효율성 개선) ---
# 프로그램 시작 시 한 번만 파일을 불러옵니다.
try:
    PRODUCT_DF = pd.read_csv('product_with_score.csv')
    INGREDIENT_DF = pd.read_csv('ingredient_data.csv')
    
    # 카테고리 이름 통일 ('로션/에멀젼' -> '로션/에멀전')
    PRODUCT_DF['카테고리'] = PRODUCT_DF['카테고리'].replace('로션/에멀젼', '로션/에멀전')
    
    # EWG 등급 딕셔너리 미리 생성
    INGREDIENT_DF.dropna(subset=['EWG등급'], inplace=True)
    EWG_DICT = pd.Series(INGREDIENT_DF['EWG등급'].values, index=INGREDIENT_DF['한국어성분명']).to_dict()
    
except FileNotFoundError:
    print("오류: 'product_with_score.csv' 또는 'ingredient_data.csv' 파일을 찾을 수 없습니다.")
    print("프로그램을 종료합니다.")
    PRODUCT_DF = None # 오류 발생 시 DataFrame을 None으로 설정

# --- 성분 리스트 정의 ---
EFFICACY_INGREDIENTS = {
    '진정': ['병풀추출물', '마데카소사이드', '판테놀', '알란토인', '약모밀추출물'],
    '미백': ['나이아신아마이드', '알부틴', '비타민C', '글루타티온'],
    '보습': ['소듐하이알루로네이트', '글리세린', '세라마이드엔피', '스쿠알란', '베타-글루칸'],
    '주름/탄력': ['아데노신', '아세틸헥사펩타이드-8', '카퍼트라이펩타이드-1', '팔미토일펜타펩타이드-4', '하이드롤라이즈드콜라겐'],
    '모공/피지': ['살리실릭애씨드', '티트리잎오일', '버지니아풍년화추출물', '나이아신아마이드', '카프릴로일살리실릭애씨드']
}

CAUTION_INGREDIENTS = {
    '민감성': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올'],
    '지성': ['미네랄오일', '시어버터', '코코넛야자일', '다이메티콘'],
    '건성': ['에탄올', '변성알코올', '살리실릭애씨드'],
    '아토피 피부': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올', '프로필렌글라이콜'] # 민감성보다 좀 더 보수적인 기준
}


def calculate_final_scores(product_df, user_skin_concerns, user_skin_type):
    """
    사용자의 피부 정보(고민-리스트, 타입)를 바탕으로 모든 제품의 최종 점수를 계산하는 함수.
    """
    
    def get_score_for_product(row):
        ingredients_str = row['전성분']
        harmfulness_score = row['유해성_점수']

        if pd.isna(ingredients_str):
            return -np.inf

        ingredients = ingredients_str.split(';')
        
        # EWG 안전성 점수
        ewg_scores = [11 - EWG_DICT.get(ing, 3.0) for ing in ingredients]
        avg_ewg_score = np.mean(ewg_scores) if ewg_scores else 0

        # 효능 점수
        efficacy_score = 0
        for concern in user_skin_concerns:
            if concern in EFFICACY_INGREDIENTS:
                efficacy_score += sum(1 for ing in ingredients if ing in EFFICACY_INGREDIENTS[concern])

        # 주의 성분 감점 ('미정'일 경우 감점 없음)
        caution_penalty = 0
        if user_skin_type in CAUTION_INGREDIENTS:
            caution_penalty = sum(2 for ing in ingredients if ing in CAUTION_INGREDIENTS[user_skin_type])
        
        # 최종 점수 합산
        final_score = avg_ewg_score + efficacy_score - harmfulness_score - caution_penalty
        return final_score

    # 원본 DataFrame을 복사하여 점수 계산 진행
    df_copy = product_df.copy()
    df_copy['최종_점수'] = df_copy.apply(get_score_for_product, axis=1)
    return df_copy

def recommend_by_selection(scored_df, selected_category):
    """선택지 기반 추천 결과를 출력하는 함수"""
    category_products = scored_df[scored_df['카테고리'] == selected_category]
    top_5_products = category_products.sort_values(by='최종_점수', ascending=False).head(5)
    
    if top_5_products.empty:
        print(f"\n죄송합니다. '{selected_category}' 카테고리에서 적합한 추천 제품을 찾지 못했습니다.")
        return
        
    print(f"\n--- 🕵️‍♀️ 당신을 위한 맞춤 '{selected_category}' 추천 Top 5 ---")
    print(top_5_products[['제품명', '브랜드명', '최종_점수']].to_string(index=False))

def recommend_by_chatbot(user_type, user_input):
    """챗봇 입력 기반 추천 결과를 출력하는 함수"""
    keyword_to_category = {
        '선크림': '선크림/로션', '선세럼': '선크림/로션', '선로션': '선크림/로션',
        '클렌징폼': '클렌징 폼', '클렌저': '클렌징 폼',
        '로션': '로션/에멀전', '에멀전': '로션/에멀전',
        '스킨': '스킨/토너', '토너': '스킨/토너',
        '세럼': '에센스/앰플/세럼', '에센스': '에센스/앰플/세럼', '앰플': '에센스/앰플/세럼',
        '크림': '크림', '마스크': '시트마스크', '밤': '밤/멀티밤'
    }

    # 챗봇 입력에서 카테고리 찾기
    found_category = None
    for keyword, category in keyword_to_category.items():
        if keyword in user_input:
            found_category = category
            break
    
    if not found_category:
        print("\n죄송합니다. 어떤 종류의 제품을 찾으시는지 파악하기 어려워요.\n('크림', '로션', '선크림' 등) 제품 종류를 명확히 말씀해주시겠어요?")
        return
        
    # 챗봇 입력에서 피부 고민 키워드 찾기
    found_concerns = []
    for concern in EFFICACY_INGREDIENTS.keys():
        if concern in user_input:
            found_concerns.append(concern)
            
    # 고민이 명확하지 않으면, 모든 고민을 대상으로 점수 계산
    if not found_concerns:
        found_concerns = list(EFFICACY_INGREDIENTS.keys())
        
    print(f"\n✅ 챗봇이 이해한 당신의 고민: {', '.join(found_concerns)}")
    print("점수를 계산 중입니다...")
    
    scored_df = calculate_final_scores(PRODUCT_DF, found_concerns, user_type)
    
    category_products = scored_df[scored_df['카테고리'] == found_category]
    top_3_products = category_products.sort_values(by='최종_점수', ascending=False).head(3)

    if top_3_products.empty:
        print(f"\n죄송합니다. '{found_category}' 카테고리에서 적합한 추천 제품을 찾지 못했습니다.")
        return

    print(f"\n--- 🤖 챗봇이 추천하는 '{found_category}' Top 3 ---")
    print(top_3_products[['제품명', '브랜드명', '최종_점수']].to_string(index=False))


def main():
    """메인 프로그램 실행 함수"""
    if PRODUCT_DF is None:
        return
        
    print("--- 당신에게 꼭 맞는 화장품을 찾아드립니다! ---")

    while True:
        print("\n어떤 방식으로 추천을 도와드릴까요?")
        print("  1. 선택지로 추천받기 (제품 종류 -> 피부 타입 -> 피부 고민)")
        print("  2. 챗봇에게 자유롭게 물어보기")
        print("  3. 종료하기")
        
        try:
            main_choice = int(input("번호를 선택해주세요: "))
            
            if main_choice == 1:
                # 1. 카테고리 선택
                all_categories = sorted(PRODUCT_DF['카테고리'].unique().tolist())
                print("\n1. 어떤 종류의 제품을 찾으시나요?")
                for i, category in enumerate(all_categories):
                    print(f"  {i+1}. {category}")
                while True:
                    try:
                        category_choice = int(input("번호를 선택해주세요: "))
                        if 1 <= category_choice <= len(all_categories):
                            selected_category = all_categories[category_choice - 1]
                            break
                        else:
                            print(f"1~{len(all_categories)} 사이의 번호를 입력해주세요.")
                    except ValueError:
                        print("유효한 번호를 입력해주세요.")

                # 2. 피부 타입 선택
                types = ['민감성', '지성', '건성', '아토피 피부', '미정(해당사항 없음)']
                print("\n2. 당신의 피부 타입을 알려주세요:")
                for i, t in enumerate(types):
                    print(f"  {i+1}. {t}")
                while True:
                    try:
                        choice = int(input("번호를 선택해주세요: "))
                        if 1 <= choice <= len(types):
                            user_type = types[choice-1]
                            break
                        else:
                            print(f"1~{len(types)} 사이의 번호를 입력해주세요.")
                    except ValueError:
                        print("유효한 번호를 입력해주세요.")

                # 3. 피부 고민 선택
                concerns = list(EFFICACY_INGREDIENTS.keys())
                print("\n3. 어떤 피부 고민을 해결하고 싶으신가요? (여러 개 선택 가능)")
                for i, concern in enumerate(concerns):
                    print(f"  {i+1}. {concern}")
                
                user_concerns = []
                while True:
                    try:
                        choices_str = input("원하는 고민의 번호를 모두 골라주세요 (예: 1 3): ")
                        choices = [int(c) for c in choices_str.split()]
                        
                        valid_choices = True
                        temp_concerns = []
                        for c in choices:
                            if 1 <= c <= len(concerns):
                                temp_concerns.append(concerns[c-1])
                            else:
                                print(f"'{c}'는 잘못된 번호입니다. 1~{len(concerns)} 사이의 번호만 입력해주세요.")
                                valid_choices = False
                                break
                        
                        if valid_choices and temp_concerns:
                            user_concerns = list(set(temp_concerns))
                            break
                        elif not temp_concerns:
                            print("하나 이상의 고민을 선택해주세요.")
                    except ValueError:
                        print("숫자와 공백만 사용하여 번호를 입력해주세요. (예: 1 3)")
                
                print(f"\n✅ 선택 정보: 카테고리='{selected_category}', 피부타입='{user_type}', 고민={user_concerns}")
                print("\n점수를 계산 중입니다. 잠시만 기다려주세요...")
                scored_df = calculate_final_scores(PRODUCT_DF, user_concerns, user_type)
                recommend_by_selection(scored_df, selected_category)

            elif main_choice == 2:
                # 챗봇 모드에서는 먼저 피부 타입만 받음
                types = ['민감성', '지성', '건성', '아토피 피부', '미정(해당사항 없음)']
                print("\n챗봇 추천을 위해, 당신의 피부 타입을 먼저 알려주세요:")
                for i, t in enumerate(types):
                    print(f"  {i+1}. {t}")
                while True:
                    try:
                        choice = int(input("번호를 선택해주세요: "))
                        if 1 <= choice <= len(types):
                            user_type_for_chatbot = types[choice-1]
                            break
                        else:
                            print(f"1~{len(types)} 사이의 번호를 입력해주세요.")
                    except ValueError:
                        print("유효한 번호를 입력해주세요.")

                user_input = input(f"\n✅ '{user_type_for_chatbot}' 피부시군요! 찾고 있는 화장품에 대해 자유롭게 말씀해주세요: ")
                recommend_by_chatbot(user_type_for_chatbot, user_input)

            elif main_choice == 3:
                print("\n프로그램을 종료합니다. 이용해주셔서 감사합니다!")
                break
            else:
                print("1, 2, 3 중 하나를 선택해주세요.")

        except ValueError:
            print("유효한 번호를 입력해주세요.")

if __name__ == '__main__':
    main()



In [26]:
import pandas as pd
import numpy as np

# --- 데이터 전역 로드 (효율성 개선) ---
# 프로그램 시작 시 한 번만 파일을 불러옵니다.
try:
    PRODUCT_DF = pd.read_csv('product_with_score.csv')
    INGREDIENT_DF = pd.read_csv('ingredient_data.csv')
    
    # 카테고리 이름 통일 ('로션/에멀젼' -> '로션/에멀전')
    PRODUCT_DF['카테고리'] = PRODUCT_DF['카테고리'].replace('로션/에멀젼', '로션/에멀전')
    
    # EWG 등급 딕셔너리 미리 생성
    INGREDIENT_DF.dropna(subset=['EWG등급'], inplace=True)
    EWG_DICT = pd.Series(INGREDIENT_DF['EWG등급'].values, index=INGREDIENT_DF['한국어성분명']).to_dict()
    
except FileNotFoundError:
    print("오류: 'product_with_score.csv' 또는 'ingredient_data.csv' 파일을 찾을 수 없습니다.")
    print("프로그램을 종료합니다.")
    PRODUCT_DF = None # 오류 발생 시 DataFrame을 None으로 설정

# --- 성분 리스트 정의 ---
EFFICACY_INGREDIENTS = {
    '진정': ['병풀추출물', '마데카소사이드', '판테놀', '알란토인', '약모밀추출물'],
    '미백': ['나이아신아마이드', '알부틴', '비타민C', '글루타티온'],
    '보습': ['소듐하이알루로네이트', '글리세린', '세라마이드엔피', '스쿠알란', '베타-글루칸'],
    '주름/탄력': ['아데노신', '아세틸헥사펩타이드-8', '카퍼트라이펩타이드-1', '팔미토일펜타펩타이드-4', '하이드롤라이즈드콜라겐'],
    '모공/피지': ['살리실릭애씨드', '티트리잎오일', '버지니아풍년화추출물', '나이아신아마이드', '카프릴로일살리실릭애씨드']
}

CAUTION_INGREDIENTS = {
    '민감성': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올'],
    '지성': ['미네랄오일', '시어버터', '코코넛야자일', '다이메티콘'],
    '건성': ['에탄올', '변성알코올', '살리실릭애씨드'],
    '아토피 피부': ['향료', '리모넨', '리날룰', '벤질알코올', '에탄올', '프로필렌글라이콜'] # 민감성보다 좀 더 보수적인 기준
}


def calculate_final_scores(product_df, user_skin_concerns, user_skin_type):
    """
    사용자의 피부 정보(고민-리스트, 타입)를 바탕으로 모든 제품의 최종 점수를 계산하는 함수.
    """
    
    def get_score_for_product(row):
        ingredients_str = row['전성분']
        harmfulness_score = row['유해성_점수']

        if pd.isna(ingredients_str):
            return -np.inf

        ingredients = ingredients_str.split(';')
        
        # EWG 안전성 점수
        ewg_scores = [11 - EWG_DICT.get(ing, 3.0) for ing in ingredients]
        avg_ewg_score = np.mean(ewg_scores) if ewg_scores else 0

        # 효능 점수
        efficacy_score = 0
        for concern in user_skin_concerns:
            if concern in EFFICACY_INGREDIENTS:
                efficacy_score += sum(1 for ing in ingredients if ing in EFFICACY_INGREDIENTS[concern])

        # 주의 성분 감점 ('미정'일 경우 감점 없음)
        caution_penalty = 0
        if user_skin_type in CAUTION_INGREDIENTS:
            caution_penalty = sum(2 for ing in ingredients if ing in CAUTION_INGREDIENTS[user_skin_type])
        
        # 최종 점수 합산
        final_score = avg_ewg_score + efficacy_score - harmfulness_score - caution_penalty
        return final_score

    # 원본 DataFrame을 복사하여 점수 계산 진행
    df_copy = product_df.copy()
    df_copy['최종_점수'] = df_copy.apply(get_score_for_product, axis=1)
    return df_copy

def recommend_by_selection(scored_df, selected_category):
    """선택지 기반 추천 결과를 출력하는 함수"""
    category_products = scored_df[scored_df['카테고리'] == selected_category]
    top_5_products = category_products.sort_values(by='최종_점수', ascending=False).head(5)
    
    if top_5_products.empty:
        print(f"\n죄송합니다. '{selected_category}' 카테고리에서 적합한 추천 제품을 찾지 못했습니다.")
        return
        
    print(f"\n--- 🕵️‍♀️ 당신을 위한 맞춤 '{selected_category}' 추천 Top 5 ---")
    print(top_5_products[['제품명', '브랜드명', '최종_점수']].to_string(index=False))

def recommend_by_chatbot(user_input):
    """챗봇 입력 기반 추천 결과를 출력하는 함수"""
    keyword_to_category = {
        '선크림': '선크림/로션', '선세럼': '선크림/로션', '선로션': '선크림/로션',
        '클렌징폼': '클렌징 폼', '클렌저': '클렌징 폼',
        '로션': '로션/에멀전', '에멀전': '로션/에멀전',
        '스킨': '스킨/토너', '토너': '스킨/토너',
        '세럼': '에센스/앰플/세럼', '에센스': '에센스/앰플/세럼', '앰플': '에센스/앰플/세럼',
        '크림': '크림', '마스크': '시트마스크', '밤': '밤/멀티밤'
    }

    # 챗봇 입력에서 카테고리 찾기
    found_category = None
    for keyword, category in keyword_to_category.items():
        if keyword in user_input:
            found_category = category
            break
    
    if not found_category:
        print("\n죄송합니다. 어떤 종류의 제품을 찾으시는지 파악하기 어려워요.\n('크림', '로션', '선크림' 등) 제품 종류를 명확히 말씀해주시겠어요?")
        return
        
    # 챗봇 입력에서 언급된 피부 고민 키워드만 찾기
    found_concerns = []
    for concern in EFFICACY_INGREDIENTS.keys():
        if concern in user_input:
            found_concerns.append(concern)
            
    if found_concerns:
        print(f"\n✅ 챗봇이 이해한 당신의 고민: {', '.join(found_concerns)}")
    else:
        print("\n✅ 특정 피부 고민이 언급되지 않아, 기본 점수로 추천해드릴게요.")
        
    print("점수를 계산 중입니다...")
    
    # 챗봇 모드에서는 피부타입을 '미정'으로 간주하여 감점 없음
    scored_df = calculate_final_scores(PRODUCT_DF, found_concerns, '미정(해당사항 없음)')
    
    category_products = scored_df[scored_df['카테고리'] == found_category]
    top_3_products = category_products.sort_values(by='최종_점수', ascending=False).head(3)

    if top_3_products.empty:
        print(f"\n죄송합니다. '{found_category}' 카테고리에서 적합한 추천 제품을 찾지 못했습니다.")
        return

    print(f"\n--- 🤖 챗봇이 추천하는 '{found_category}' Top 3 ---")
    print(top_3_products[['제품명', '브랜드명', '최종_점수']].to_string(index=False))


def main():
    """메인 프로그램 실행 함수"""
    if PRODUCT_DF is None:
        return
        
    print("--- 당신에게 꼭 맞는 화장품을 찾아드립니다! ---")

    while True:
        print("\n어떤 방식으로 추천을 도와드릴까요?")
        print("  1. 선택지로 추천받기 (제품 종류 -> 피부 타입 -> 피부 고민)")
        print("  2. 챗봇에게 자유롭게 물어보기")
        print("  3. 종료하기")
        
        try:
            main_choice = int(input("번호를 선택해주세요: "))
            
            if main_choice == 1:
                # 1. 카테고리 선택
                all_categories = sorted(PRODUCT_DF['카테고리'].unique().tolist())
                print("\n1. 어떤 종류의 제품을 찾으시나요?")
                for i, category in enumerate(all_categories):
                    print(f"  {i+1}. {category}")
                while True:
                    try:
                        category_choice = int(input("번호를 선택해주세요: "))
                        if 1 <= category_choice <= len(all_categories):
                            selected_category = all_categories[category_choice - 1]
                            break
                        else:
                            print(f"1~{len(all_categories)} 사이의 번호를 입력해주세요.")
                    except ValueError:
                        print("유효한 번호를 입력해주세요.")

                # 2. 피부 타입 선택
                types = ['민감성', '지성', '건성', '아토피 피부', '미정(해당사항 없음)']
                print("\n2. 당신의 피부 타입을 알려주세요:")
                for i, t in enumerate(types):
                    print(f"  {i+1}. {t}")
                while True:
                    try:
                        choice = int(input("번호를 선택해주세요: "))
                        if 1 <= choice <= len(types):
                            user_type = types[choice-1]
                            break
                        else:
                            print(f"1~{len(types)} 사이의 번호를 입력해주세요.")
                    except ValueError:
                        print("유효한 번호를 입력해주세요.")

                # 3. 피부 고민 선택
                concerns = list(EFFICACY_INGREDIENTS.keys())
                print("\n3. 어떤 피부 고민을 해결하고 싶으신가요? (여러 개 선택 가능)")
                for i, concern in enumerate(concerns):
                    print(f"  {i+1}. {concern}")
                
                user_concerns = []
                while True:
                    try:
                        choices_str = input("원하는 고민의 번호를 모두 골라주세요 (예: 1 3): ")
                        choices = [int(c) for c in choices_str.split()]
                        
                        valid_choices = True
                        temp_concerns = []
                        for c in choices:
                            if 1 <= c <= len(concerns):
                                temp_concerns.append(concerns[c-1])
                            else:
                                print(f"'{c}'는 잘못된 번호입니다. 1~{len(concerns)} 사이의 번호만 입력해주세요.")
                                valid_choices = False
                                break
                        
                        if valid_choices and temp_concerns:
                            user_concerns = list(set(temp_concerns))
                            break
                        elif not temp_concerns:
                            print("하나 이상의 고민을 선택해주세요.")
                    except ValueError:
                        print("숫자와 공백만 사용하여 번호를 입력해주세요. (예: 1 3)")
                
                print(f"\n✅ 선택 정보: 카테고리='{selected_category}', 피부타입='{user_type}', 고민={user_concerns}")
                print("\n점수를 계산 중입니다. 잠시만 기다려주세요...")
                scored_df = calculate_final_scores(PRODUCT_DF, user_concerns, user_type)
                recommend_by_selection(scored_df, selected_category)

            elif main_choice == 2:
                user_input = input("\n찾고 있는 화장품에 대해 자유롭게 말씀해주세요: ")
                recommend_by_chatbot(user_input)

            elif main_choice == 3:
                print("\n프로그램을 종료합니다. 이용해주셔서 감사합니다!")
                break
            else:
                print("1, 2, 3 중 하나를 선택해주세요.")

        except ValueError:
            print("유효한 번호를 입력해주세요.")

if __name__ == '__main__':
    main()


--- 당신에게 꼭 맞는 화장품을 찾아드립니다! ---

어떤 방식으로 추천을 도와드릴까요?
  1. 선택지로 추천받기 (제품 종류 -> 피부 타입 -> 피부 고민)
  2. 챗봇에게 자유롭게 물어보기
  3. 종료하기

프로그램을 종료합니다. 이용해주셔서 감사합니다!
